In [1]:
%matplotlib inline
import glob, os, sys
from astropy import coordinates as coords
from astropy import units as u
from astropy import constants as const

from astropy.io import fits
from astropy.io.fits import getdata
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st

import warnings
warnings.filterwarnings("ignore")
plt.rcParams['axes.linewidth'] = 1.5

In [2]:
band = 'g'

### Read Light Curve Data

In [3]:
# Read light curves
path = 'data/y6a1_cdfs_light_curves.csv'
df_lc_psf = pd.read_csv(path)
# Lower-case column names
df_lc_psf.columns = df_lc_psf.columns.str.lower()
df_lc_psf = df_lc_psf[df_lc_psf['band']==band]
df_lc_psf['roms'] = [-1]*len(df_lc_psf)
# Combine on RA, dec (effectively match to 0.5 arcsec)
df_lc_psf['ra_as'] = np.around(df_lc_psf['ra']*3600*2,0)/2.0 # deg->arcsec
df_lc_psf['dec_as'] = np.around(df_lc_psf['dec']*3600*2,0)/2.0 # deg->arcsec
coord_unique = np.unique(np.array([df_lc_psf['ra_as'],df_lc_psf['dec_as']]).T,axis=0)
# Create SkyCoord
coord_des_psf = SkyCoord(coord_unique[:,0],coord_unique[:,1],unit=u.arcsec)

In [4]:
df_lc_psf

,ra,dec,mjd_obs,flux_psf,fluxerr_psf,spread_model,band,roms,ra_as,dec_as
0,52.986910,-27.664296,56536.257124,17.718875,0.474467,-0.001892,g,-1,190753.0,-99591.5
1,52.998971,-27.680238,56536.257124,0.000000,0.000000,-0.006526,g,-1,190796.5,-99649.0
2,52.998463,-27.753025,56536.257124,3.328042,0.466259,0.001849,g,-1,190794.5,-99911.0
3,52.982354,-27.680166,56536.257124,4.410336,0.437443,0.009430,g,-1,190736.5,-99648.5
4,52.989296,-27.740097,56536.257124,2.760407,0.720181,-0.039473,g,-1,190761.5,-99864.5
...,...,...,...,...,...,...,...,...,...,...
13779179,53.151799,-27.775491,58454.271911,0.231750,0.036128,0.004247,g,-1,191346.5,-99992.0
13779180,53.263503,-27.875838,58454.271911,0.435692,0.034904,0.004473,g,-1,191748.5,-100353.0
13779181,53.264301,-27.802016,58454.271911,0.163402,0.033040,0.022134,g,-1,191751.5,-100087.5
13779182,53.264415,-27.793660,58454.271911,0.460810,0.034266,0.009817,g,-1,191752.0,-100057.0


./desdia -t DES0332-2749 -p supernova -n 30 -w /data/des80.a/data/cburke/

In [12]:
# Read light curves
path = 'data/dia/DES0251+0043.dat'
df_lc_dia = pd.read_csv(path,sep='\s+',escapechar='#')
# Lower-case column names
df_lc_dia.columns = df_lc_dia.columns.str.lower()
df_lc_dia['roms'] = [-1]*len(df_lc_dia)
# Combine on num
idx_dia = np.unique(df_lc_dia['num'])
#idx = df_lc_dia.index.get_level_values('num').unique()
ras = [df_lc_dia['ra'][i] for i in idx_dia]
decs = [df_lc_dia['dec'][i] for i in idx_dia]
# Create SkyCoord
coord_des_dia = SkyCoord(ras,decs,unit=u.deg)

In [13]:
df_lc_dia

,num,mjd,ra,dec,flux3,flux4,flux5,fluxerr3,fluxerr4,fluxerr5,roms
0,1,56325.068647,42.739463,0.369453,17.479680,17.306115,17.179451,0.003205,0.003357,0.003347,-1
1,2,56325.068647,42.731207,0.367415,19.121040,18.641280,18.301826,0.005098,0.004271,0.003775,-1
2,3,56325.068647,42.731485,0.366869,18.946807,18.462798,18.054244,0.004766,0.003927,0.003184,-1
3,4,56325.068647,43.094327,0.366515,21.716462,21.611195,21.593013,0.028596,0.036340,0.045466,-1
4,5,56325.068647,42.533103,0.365960,NaN,NaN,NaN,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...
51511,2858,58453.123097,42.846782,1.070109,NaN,NaN,NaN,NaN,NaN,NaN,-1
51512,2859,58453.123097,42.459419,1.057550,NaN,NaN,NaN,NaN,NaN,NaN,-1
51513,2860,58453.123097,43.084822,1.083625,NaN,NaN,NaN,NaN,NaN,NaN,-1
51514,2861,58453.123097,42.630266,1.058391,NaN,NaN,NaN,NaN,NaN,NaN,-1


In [16]:
idx_dia_psf

array([], dtype=int64)

### Match DIA/SAP

In [15]:
idx,d2d,d3d = coord_des_psf.match_to_catalog_sky(coord_des_dia)
match = d2d < 0.5*u.arcsec
idx_dia_psf = idx_dia[idx[match]]
coord_psf_dia = coord_des_psf[match]
print('Found %d matches.' % len(match))

Found 284694 matches.


### Fix Error Bars

In [25]:
df_lc_psf['fluxerr_psf_corr'] = df_lc_psf['fluxerr_psf']*0.9*(df_lc_psf['fluxerr_psf']/df_lc_psf['flux_psf'])**-0.1

### Define Selection functions

We adopt the metric reduced median spread (ROMS), based on the robust median absolute deviation sstatistic:

$$ \rm{ROMS} = median( | \rm{flux}_i - \rm{median} | /  \rm{flux\_error}_i ) $$

We adopt $\rm{ROMS}>2$ as our AGN candidate sample. See https://opensource.ncsa.illinois.edu/confluence/display/DESDM/Y3A1+SN+Field+Variability+Catalog.

In [26]:
def reject_outliers(data, m=3.0):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.0
    return data[s<m]

def plot_lc(mjd, flux, fluxerr):
    fig, ax = plt.subplots(figsize=(8,4))
    mag = 22.5 -2.5*np.log10(flux)
    magerr = 2.5/np.log(10)*fluxerr/flux
    ax.errorbar(mjd,mag,yerr=magerr,c='k',fmt='.')
    ax.set_xlabel('Time (MJD)',fontsize=18)
    ax.set_ylabel(r'Magnitude $g$',fontsize=18)
    ax.tick_params(labelsize=18)
    #ax.set_ylim(np.nanmax(mag)+.1, np.nanmin(mag)-.1)
    fig.tight_layout()
    return ax

def select(ra, dec, m=3.0, v=3.0, plot=True):
    # Get rows of object
    ra_as = np.around(ra*2,0)/2.0 # deg->arcsec
    dec_as = np.around(dec*2,0)/2.0 # deg->arcsec
    ind = (df_lc['ra_as']==ra_as) & (df_lc['dec_as']==dec_as)
    dfi = df_lc[ind]
    # Get light curve
    mjd = dfi['mjd_obs'].values
    flux = dfi['flux_psf'].values
    fluxerr = dfi['fluxerr_psf_corr'].values
    # Bad flux
    mask = flux>0
    mjd = mjd[mask]
    flux = flux[mask]
    fluxerr = fluxerr[mask]
    # Reject outliers
    d = np.abs(flux - np.nanmedian(flux))
    mdev = np.nanmedian(d)
    s = d/mdev if mdev else 0.
    mjd = mjd[s<m]
    flux = flux[s<m]
    fluxerr = fluxerr[s<m]
    # Number of epochs
    n_epochs = len(mjd)
    # Magnitude cut
    med_mag = np.nanmedian(22.5 -2.5*np.log10(flux))
    # Spread_model cut (unresolved sources only)
    if np.median(dfi['spread_model'].values) > 0.003:
        return -2.0, med_mag, n_epochs
    if med_mag < 17.0:
        return -2.0, med_mag, n_epochs
    n_epochs = len(mjd)
    # Compute reduced median spread (ROMS) in units of sigma
    roms = 1.48*np.nanmedian(np.abs(flux - np.nanmedian(flux))/fluxerr)
    # Add sigma_var to DataFrame
    df_lc['roms'][ind] = roms
    # Plot
    if plot and roms > v and n_epochs>5:
        ax = plot_lc(mjd,flux,fluxerr)
        ax.text(.05,.9,r'$\sigma_{\rm{var}}=%.1f$' % roms,transform=ax.transAxes,fontsize=18)
    return roms, med_mag, n_epochs

In [11]:
coord_psf_dia

<SkyCoord (ICRS): (ra, dec) in deg
    []>

In [10]:
vselect = np.vectorize(select)
roms, mag, numepoch = vselect(coord_psf_dia.ra.arcsec, coord_psf_dia.dec.arcsec, m=3.0, v=3.0, plot=True)

NameError: name 'select' is not defined